In [3]:
from __future__ import print_function, division
import numpy as np
import os
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
from tqdm import tqdm
import keras as K
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold as KFold
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier as RF
import xgboost as xgb
import pickle
import torch

In [4]:
cd /home/chenxupeng/projects/pr/

/home/chenxupeng/projects/pr


In [50]:
annotation = pd.read_csv('data/annotations.csv')
annotation_random_ = pd.read_csv('data/csvfiles/annotations_idrandom.csv')
candidate = pd.read_csv('data/csvfiles/candidates.csv')

In [12]:
annotation.head(10)

,Unnamed: 0,seriesuid,coordX,coordY,coordZ,diameter_mm,node
0,0,LKDS-00001,-76.449879,-49.540571,229.500000,14.180405,LKDS-00001_1
1,1,LKDS-00003,-82.905787,106.263573,226.224976,5.200858,LKDS-00003_1
2,2,LKDS-00003,-92.705387,191.515214,228.724976,6.723286,LKDS-00003_2
3,3,LKDS-00004,-68.270833,209.412809,167.229885,22.517480,LKDS-00004_1
4,4,LKDS-00005,-147.952967,-164.169479,-273.434241,5.898208,LKDS-00005_1
5,5,LKDS-00007,-36.303583,68.367853,18.875000,6.347767,LKDS-00007_1
6,6,LKDS-00011,79.240951,105.062110,-170.126775,5.838910,LKDS-00011_1
7,7,LKDS-00013,-42.644951,-86.651939,-139.800002,15.884164,LKDS-00013_1
8,8,LKDS-00015,-61.142430,-93.778010,-3.200001,8.262994,LKDS-00015_1
9,9,LKDS-00016,-127.532989,-29.607902,-253.697616,8.269325,LKDS-00016_1


In [13]:
annotation_random.head(10)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150
5,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,90.931713,149.027266,-426.544715,18.208570
6,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,89.540769,196.405159,-515.073322,16.381276
7,1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028...,81.509646,54.957219,-150.346423,10.362321
8,1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408...,105.055792,19.825260,-91.247251,21.089619
9,1.3.6.1.4.1.14519.5.2.1.6279.6001.104562737760...,-124.834262,127.247155,-473.064479,10.465854


In [14]:
annotation_random.shape

(1186, 5)

In [ ]:
#对应起来两个文件

In [19]:
count = 0
for i in range(annotation_random.shape[0]):
    dia = annotation_random['diameter_mm'][i]
    for j in range(annotation.shape[0]):
        if dia ==annotation['diameter_mm'][j]:
            count+=1
            print (i,j)
print (count)

2 922
8 463
16 464
24 956
27 281
31 838
32 839
33 840
48 9
49 10
50 11
51 767
52 768
53 769
54 499
55 500
57 637
58 638
59 639
60 640
61 641
65 844
66 845
67 917
70 37
71 383
72 26
73 27
76 611
77 490
78 491
79 492
80 493
81 494
82 495
85 350
90 591
91 592
92 593
93 594
94 595
98 187
99 39
100 40
106 586
107 587
110 877
111 231
112 951
113 952
114 953
115 954
116 34
120 703
121 704
122 412
127 333
128 334
129 911
130 912
131 913
132 914
139 596
140 878
141 310
142 311
143 312
149 72
151 563
152 501
153 502
154 503
162 32
163 156
168 785
169 786
174 50
175 51
176 52
177 53
178 54
179 55
182 159
183 160
184 169
190 942
191 943
192 944
193 945
194 946
199 938
209 124
210 125
211 126
214 968
215 657
216 658
217 659
223 484
224 485
228 452
237 184
239 57
240 58
241 59
242 60
243 61
248 314
249 315
250 316
251 317
252 318
253 319
254 320
255 608
256 609
257 690
258 691
259 692
260 66
261 67
268 747
269 748
270 749
282 374
283 375
284 811
285 812
286 813
287 814
288 815
289 816
290 817
291 81

In [20]:
#共505个一一对应的，可以找到第二个表里的，并改写

In [23]:
index = np.ndarray([505,2],dtype=np.int32)
count = 0
for i in range(annotation_random.shape[0]):
    dia = annotation_random['diameter_mm'][i]
    for j in range(annotation.shape[0]):
        if dia ==annotation['diameter_mm'][j]:
            index[count,0] = i
            index[count,1] = j
            count+=1

In [29]:
index

array([[   2,  922],
       [   8,  463],
       [  16,  464],
       ..., 
       [1169,  551],
       [1170,  552],
       [1171,  874]], dtype=int32)

In [ ]:
#利用index找到segment可用的表  即先用第二列的对应的名字命名第一列的行的 再只留下第一列的那些行

In [30]:
for i in range(505):
    annotation_random['seriesuid'][index[i][0]] = annotation['seriesuid'][index[i][1]]

/home/chenxupeng/apps/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [41]:
select = index[:,:1].flatten()

In [44]:
annotation_random.iloc[select].to_csv('data/segment_select.csv')

In [45]:
segment = pd.read_csv('data/segment_select.csv')

In [66]:
annotation_random_.iloc[8]

seriesuid      1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408...
coordX                                                   105.056
coordY                                                   19.8253
coordZ                                                  -91.2473
diameter_mm                                              21.0896
Name: 8, dtype: object

In [67]:
annotation.iloc[463]

Unnamed: 0              463
seriesuid        LKDS-00512
coordX              105.056
coordY              19.8253
coordZ             -91.2473
diameter_mm         21.0896
node           LKDS-00512_1
Name: 463, dtype: object

In [58]:
segment['random_id'] = annotation_random_['seriesuid'][index[:,0:1].flatten()].reset_index(np.arange(505))['seriesuid']

In [68]:
segment.to_csv('data/segment_select.csv')

In [69]:
segment = pd.read_csv('data/segment_select.csv')

In [71]:
segment.head(5)

,Unnamed: 0,Unnamed: 0.1,seriesuid,coordX,coordY,coordZ,diameter_mm,random_id
0,0,2,LKDS-00939,69.639017,-140.944586,876.374496,5.786348,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...
1,1,8,LKDS-00512,105.055792,19.825260,-91.247251,21.089619,1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408...
2,2,16,LKDS-00513,-98.135977,-72.867792,-221.822772,18.127908,1.3.6.1.4.1.14519.5.2.1.6279.6001.107109359065...
3,3,24,LKDS-00981,42.574154,90.267552,-84.811060,5.916174,1.3.6.1.4.1.14519.5.2.1.6279.6001.108231420525...
4,4,27,LKDS-00302,-38.096020,-106.705434,-139.548467,13.683746,1.3.6.1.4.1.14519.5.2.1.6279.6001.110678335949...


In [ ]:
#用这个来改名

In [72]:
cd data/seg-lungs-LUNA16/

/home/chenxupeng/projects/pr/data/seg-lungs-LUNA16


In [74]:
os.rename('a.txt', 'b.txt')

In [75]:
segment["random_id"][0]

'1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208'

In [85]:
from pathlib import Path

In [86]:
for i in range(505):
    my_file = Path(segment["random_id"][i]+'.mhd')
    if my_file.is_file():
        os.rename(segment["random_id"][i]+'.mhd',segment['seriesuid'][i]+'.mhd')
        os.rename(segment["random_id"][i]+'.zraw',segment['seriesuid'][i]+'.zraw')

In [79]:
segment["random_id"][i]+'.mhd'

'1.3.6.1.4.1.14519.5.2.1.6279.6001.111496024928645603833332252962.mhd'

In [1]:
#查看segment

In [2]:
from __future__ import print_function, division
import numpy as np
import os
import csv
from glob import glob
import pandas as pd
import numpy as np
import SimpleITK as sitk
from skimage import measure,morphology
from sklearn.cluster import KMeans
from skimage.transform import resize
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
from tqdm import tqdm

In [3]:
cd /home/chenxupeng/projects/pr/

/home/chenxupeng/projects/pr


In [4]:
luna_subset_path = 'data/useful_seg/'
file_list=glob(luna_subset_path+"*.mhd")

In [5]:
def get_filename(case):
    global file_list
    for f in file_list:
        if case in f:
            return(f)

In [6]:
# The locations of the nodes
df_node = pd.read_csv("data/segment_select.csv")
df_node["file"] = df_node["seriesuid"].apply(get_filename)
df_node = df_node.dropna()

In [11]:
df_node.iloc[:,2:]

,seriesuid,coordX,coordY,coordZ,diameter_mm,random_id,file
0,LKDS-00939,69.639017,-140.944586,876.374496,5.786348,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,data/useful_seg/LKDS-00939.mhd
1,LKDS-00512,105.055792,19.825260,-91.247251,21.089619,1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408...,data/useful_seg/LKDS-00512.mhd
2,LKDS-00513,-98.135977,-72.867792,-221.822772,18.127908,1.3.6.1.4.1.14519.5.2.1.6279.6001.107109359065...,data/useful_seg/LKDS-00513.mhd
3,LKDS-00981,42.574154,90.267552,-84.811060,5.916174,1.3.6.1.4.1.14519.5.2.1.6279.6001.108231420525...,data/useful_seg/LKDS-00981.mhd
4,LKDS-00302,-38.096020,-106.705434,-139.548467,13.683746,1.3.6.1.4.1.14519.5.2.1.6279.6001.110678335949...,data/useful_seg/LKDS-00302.mhd
5,LKDS-00866,100.198500,28.864418,-212.788054,5.190274,1.3.6.1.4.1.14519.5.2.1.6279.6001.111496024928...,data/useful_seg/LKDS-00866.mhd
6,LKDS-00866,98.790467,9.538893,-183.240431,3.857519,1.3.6.1.4.1.14519.5.2.1.6279.6001.111496024928...,data/useful_seg/LKDS-00866.mhd
7,LKDS-00866,15.765133,83.862481,-146.098822,5.111034,1.3.6.1.4.1.14519.5.2.1.6279.6001.111496024928...,data/useful_seg/LKDS-00866.mhd
8,LKDS-00016,-127.532989,-29.607902,-253.697616,8.269325,1.3.6.1.4.1.14519.5.2.1.6279.6001.114218724025...,data/useful_seg/LKDS-00016.mhd
9,LKDS-00016,93.138882,-5.652615,-213.910615,6.209760,1.3.6.1.4.1.14519.5.2.1.6279.6001.114218724025...,data/useful_seg/LKDS-00016.mhd


In [12]:
# Looping over the image files
#just use the biggest node!
fcount = 0
for img_file in file_list:
    #print "Getting mask for image file %s" % img_file.replace(luna_subset_path,"")
    mini_df = df_node[df_node["file"]==img_file] #get all nodules associate with file
    if len(mini_df)>0:       # some files may not have a nodule--skipping those 
        biggest_node = np.argsort(mini_df["diameter_mm"].values)[-1]   # just using the biggest node
        node_x = mini_df["coordX"].values[biggest_node]
        node_y = mini_df["coordY"].values[biggest_node]
        node_z = mini_df["coordZ"].values[biggest_node]
        diam = mini_df["diameter_mm"].values[biggest_node]

In [13]:
img_file

'data/useful_seg/LKDS-00998.mhd'

In [14]:
#get node position
itk_img = sitk.ReadImage(img_file) 
img_array = sitk.GetArrayFromImage(itk_img) # indexes are z,y,x (notice the ordering)
center = np.array([node_x,node_y,node_z])   # nodule center
origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
spacing = np.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
v_center =np.rint((center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)

RuntimeError: Exception thrown in SimpleITK ReadImage: /tmp/SimpleITK-build/ITK/Modules/IO/Meta/src/itkMetaImageIO.cxx:483:
itk::ERROR: MetaImageIO(0x3dc45c0): File cannot be read: data/useful_seg/LKDS-00998.mhd for reading.
Reason: Success

In [19]:
itk_img = sitk.ReadImage('data/useful_seg/LKDS-00998.mhd') 
sitk.GetArrayFromImage(itk_img)

RuntimeError: Exception thrown in SimpleITK ReadImage: /tmp/SimpleITK-build/ITK/Modules/IO/Meta/src/itkMetaImageIO.cxx:483:
itk::ERROR: MetaImageIO(0x3e06f70): File cannot be read: data/useful_seg/LKDS-00998.mhd for reading.
Reason: Success